In [1]:
import nest_asyncio
# Allows nested asyncio event loops - Jupyter already runs in an event loop,
# so nest_asyncio lets us use asyncio.run() within this environment
nest_asyncio.apply()

In [2]:
from haystack import Pipeline
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.tools import ToolInvoker
from haystack.components.converters import OutputAdapter
from haystack.dataclasses import ChatMessage

In [3]:
from haystack_integrations.tools.mcp.mcp_tool import SSEServerInfo
from mcp_server import MCPServer

In [4]:
import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key:  ········


In [9]:
google_maps_mcp = MCPServer(server_info=SSEServerInfo(base_url=f"http://localhost:8000"))

Error in sse_reader: peer closed connection without sending complete message body (incomplete chunked read)
Error in sse_reader: peer closed connection without sending complete message body (incomplete chunked read)
Error in sse_reader: peer closed connection without sending complete message body (incomplete chunked read)
Error in sse_reader: peer closed connection without sending complete message body (incomplete chunked read)
Error in sse_reader: peer closed connection without sending complete message body (incomplete chunked read)
Error in sse_reader: peer closed connection without sending complete message body (incomplete chunked read)
Error in sse_reader: peer closed connection without sending complete message body (incomplete chunked read)


In [10]:
pipeline = Pipeline()
pipeline.add_component("llm", OpenAIChatGenerator(model="gpt-4o", tools=google_maps_mcp))
pipeline.add_component("tool_invoker", ToolInvoker(tools=google_maps_mcp))
pipeline.add_component(
    "adapter",
    OutputAdapter(
        template="{{ initial_msg + initial_tool_messages + tool_messages }}",
        output_type=list[ChatMessage],
        unsafe=True,
    ),
)
pipeline.add_component("response_llm", OpenAIChatGenerator(model="gpt-4o-mini"))


pipeline.connect("llm.replies", "tool_invoker.messages")
pipeline.connect("llm.replies", "adapter.initial_tool_messages")
pipeline.connect("tool_invoker.tool_messages", "adapter.tool_messages")
pipeline.connect("adapter.output", "response_llm.messages")

Unsafe mode is enabled. This allows execution of arbitrary code in the Jinja template. Use this only if you trust the source of the template.


🚅 Components
  - llm: OpenAIChatGenerator
  - tool_invoker: ToolInvoker
  - adapter: OutputAdapter
  - response_llm: OpenAIChatGenerator
🛤️ Connections
  - llm.replies -> tool_invoker.messages (List[ChatMessage])
  - llm.replies -> adapter.initial_tool_messages (List[ChatMessage])
  - tool_invoker.tool_messages -> adapter.tool_messages (List[ChatMessage])
  - adapter.output -> response_llm.messages (list[ChatMessage])

In [11]:
def query_pipeline(user_input):
    user_input_msg = ChatMessage.from_user(text=user_input)
    result = pipeline.run({
        "llm": {"messages": [user_input_msg]}, 
        "adapter": {"initial_msg": [user_input_msg]}
    }, include_outputs_from={"tool_invoker"})
    
    print("\nUser:", user_input)
    print("\nAssistant:", result["response_llm"]["replies"][0].text)
    print("\nTool invocations:")
    for tool_call in result["tool_invoker"].get("tool_calls", []):
        print(f"  - Tool: {tool_call.get('name')}")
        print(f"    Arguments: {tool_call.get('arguments')}")
        print(f"    Response: {tool_call.get('response')}")
    
    return result

In [12]:
query_pipeline(user_input = "Which city has higher elevation, Frankurt or Paris?")


User: Which city has higher elevation, Frankurt or Paris?

Assistant: To compare the elevations:

- **Frankfurt, Germany**: The city is situated at an elevation of about 112 meters (367 feet) above sea level.
- **Paris, France**: The city has a lower elevation of around 35 meters (115 feet) above sea level.

Therefore, **Frankfurt** has a higher elevation than **Paris**.

Tool invocations:


{'tool_invoker': {'tool_messages': [ChatMessage(_role=<ChatRole.TOOL: 'tool'>, _content=[ToolCallResult(result='meta=None content=[TextContent(type=\'text\', text=\'{\\n  "places": [\\n    {\\n      "name": "Frankfurt am Main",\\n      "formatted_address": "Frankfurt am Main, Germany",\\n      "location": {\\n        "lat": 50.1109221,\\n        "lng": 8.6821267\\n      },\\n      "place_id": "ChIJxZZwR28JvUcRAMawKVBDIgQ",\\n      "types": [\\n        "locality",\\n        "political"\\n      ]\\n    }\\n  ]\\n}\', annotations=None)] isError=False', origin=ToolCall(tool_name='maps_search_places', arguments={'query': 'Frankfurt, Germany'}, id='call_zyvP8iGaVcGUatNQtwd3ZyKB'), error=False)], _name=None, _meta={}),
   ChatMessage(_role=<ChatRole.TOOL: 'tool'>, _content=[ToolCallResult(result='meta=None content=[TextContent(type=\'text\', text=\'{\\n  "places": [\\n    {\\n      "name": "Paris",\\n      "formatted_address": "Paris, France",\\n      "location": {\\n        "lat": 48.8575475